In [9]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [2]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image
def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
## Right lane, negative slope
## Left lane, positive slope
    x_Nve,y_Nve,x_Pve,y_Pve,slope_Nve,slope_Pve,intercept_Nve,intercept_Pve,Y,X = [],[],[],[],[],[],[],[],[],[]
    y_min = 310
    y_max = img.shape[0]
    for line in lines:
        for x1,y1,x2,y2 in line:
            #identify global minimum y
            if y1<310:
                y_min=y1
                eu_dist = math.hypot(x2 - x1, y2 - y1)
            if x2 != x1:
                slope = (y2-y1)/(x2-x1)
        #intercept = np.mean([(y1 - slope*x1),(y2-slope*x2)])
            if 0.8 > slope > 0.4:
                slope_Pve.append(slope)
                x_Pve.append(x1)
                x_Pve.append(x2)
                y_Pve.append(y1)
                y_Pve.append(y2)
            #intercept_Pve.append(intercept)
            if -0.8 < slope < -0.4:
                slope_Nve.append(slope)
                x_Nve.append(x1)
                x_Nve.append(x2)
                y_Nve.append(y1)
                y_Nve.append(y2)  
            #intercept_Nve.append(intercept)
   
    #Averaging the position of line segment right side
    avg_slope_Nve= sum(slope_Nve)/float(len(slope_Nve))
    avg_x_Nve=sum(x_Nve)/float(len(x_Nve))
    avg_y_Nve=sum(y_Nve)/float(len(y_Nve))
    #Calculate intercept for averaged right coordinates
    intercept_Nve= avg_y_Nve-(avg_slope_Nve*avg_x_Nve)
    #Find x1,x2 for highest point, we already have y1,y2
    x1_Nve=(y_min-intercept_Nve)/avg_slope_Nve
    x2_Nve=(y_max-intercept_Nve)/avg_slope_Nve
    if (math.isnan(x1_Nve) or math.isnan(x2_Nve)):
        return
    #Draw right line
    cv2.line(img, (int(x1_Nve), y_min), (int(x2_Nve), y_max),[0,0,175],20)
    
    #Averaging the position of line segment left side
    avg_slope_Pve= sum(slope_Pve)/float(len(slope_Pve))
    avg_x_Pve=sum(x_Pve)/float(len(x_Pve))
    avg_y_Pve=sum(y_Pve)/float(len(y_Pve))
    #Calculate intercept for averaged left coordinates
    intercept_Pve= avg_y_Pve-(avg_slope_Pve*avg_x_Pve)
    #Find x1,x2 for highest point, we already have y1,y2
    x1_Pve=(y_min-intercept_Pve)/avg_slope_Pve
    x2_Pve=(y_max-intercept_Pve)/avg_slope_Pve
    if (math.isnan(x1_Pve) or math.isnan(x2_Pve)):
        return
    #Draw left line
    cv2.line(img, (int(x1_Pve), y_min), (int(x2_Pve), y_max),[0,0,175],20)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [13]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image
def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
## Right lane, negative slope
## Left lane, positive slope
    x_Nve,y_Nve,x_Pve,y_Pve,slope_Nve,slope_Pve,intercept_Nve,intercept_Pve,Y,X = [],[],[],[],[],[],[],[],[],[]
    y_min = 310
    y_max = img.shape[0]
    count_Nve=1
    count_Pve=1
    for line in lines:
        for x1,y1,x2,y2 in line:
            #identify global minimum y
            if y1<310:
                y_min=y1
                eu_dist = math.hypot(x2 - x1, y2 - y1)
            if x2 != x1:
                slope = (y2-y1)/(x2-x1)
        #intercept = np.mean([(y1 - slope*x1),(y2-slope*x2)])
            if 0.8 > slope > 0.4:
                count_Pve +=1
                slope_Pve.append(slope)
                x_Pve.append(x1)
                x_Pve.append(x2)
                y_Pve.append(y1)
                y_Pve.append(y2)
            #intercept_Pve.append(intercept)
            if -0.8 < slope < -0.4:
                count_Nve +=1
                slope_Nve.append(slope)
                x_Nve.append(x1)
                x_Nve.append(x2)
                y_Nve.append(y1)
                y_Nve.append(y2)  
            #intercept_Nve.append(intercept)
   
    #Averaging the position of line segment right side
    if count_Nve !=0:
        avg_slope_Nve= sum(slope_Nve)/(count_Nve-1)
        avg_x_Nve=sum(x_Nve)/((count_Nve-1)*2)
        avg_y_Nve=sum(y_Nve)/((count_Nve-1)*2)
    #Calculate intercept for averaged right coordinates
        intercept_Nve= avg_y_Nve-(avg_slope_Nve*avg_x_Nve)
    #Find x1,x2 for highest point, we already have y1,y2
    #if (math.isnan(x1_Nve) or math.isnan(x2_Nve)): 
    x1_Nve=(y_min-intercept_Nve)/avg_slope_Nve
    x2_Nve=(y_max-intercept_Nve)/avg_slope_Nve
    #Draw right line
    if y_min-intercept_Nve!=0 or y_max-intercept_Nve!=0 or math.isnan(intercept_Nve) or math.isinf(intercept_Nve):
        x1_Nve=(y_min-intercept_Nve)/avg_slope_Nve
        x2_Nve=(y_max-intercept_Nve)/avg_slope_Nve
    cv2.line(img, (int(x1_Nve), y_min), (int(x2_Nve), y_max),[0,0,175],20)
    
    #Averaging the position of line segment left side
    if count_Pve !=0:
        avg_slope_Pve= float(sum(slope_Pve))/(count_Pve-1)
        avg_x_Pve=sum(x_Pve)/((count_Pve-1)*2)
        avg_y_Pve=sum(y_Pve)/((count_Pve-1)*2)
    #Calculate intercept for averaged left coordinates
        intercept_Pve= avg_y_Pve-(avg_slope_Pve*avg_x_Pve)
    #Find x1,x2 for highest point, we already have y1,y2
    #if (math.isnan(x1_Pve) or math.isnan(x2_Pve)):
    x1_Pve=(y_min-intercept_Pve)/avg_slope_Pve
    x2_Pve=(y_max-intercept_Pve)/avg_slope_Pve
    #Draw left line
    #Find x1,x2 for highest point, we already have y1,y2
    if y_min-intercept_Pve!=0 or y_max-intercept_Pve!=0 or math.isnan(intercept_Pve) or math.isinf(intercept_Pve):
        x1_Pve=(y_min-intercept_Pve)/avg_slope_Pve
        x2_Pve=(y_max-intercept_Pve)/avg_slope_Pve
    cv2.line(img, (int(x1_Pve), y_min), (int(x2_Pve), y_max),[0,0,175],20)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

In [ ]:
def process_test_image(image):
    for i in range(len(image)):
        img = (mpimg.imread("test_images/"+image[i])) #original image
        gray=grayscale(img)
        blur_gray =gaussian_blur(gray,3)
        edges=canny(blur_gray,75,150)
        masked_edges=region_of_interest(edges,np.array([[(150,540),(450,320),(490,310),(900,540)]]))
        h_img=hough_lines(masked_edges,2,np.pi/180,15,40,20)
        final=weighted_img(h_img,img)
        plt.imshow(final)
        plt.show()

In [ ]:
import os
images=os.listdir("test_images/")
#process_test_image(images)

## Movie

In [14]:
def process_image(image):
    gray=grayscale(image)
    blur_gray =gaussian_blur(gray,3)
    edges=canny(blur_gray,75,150)
    masked_edges=region_of_interest(edges,np.array([[(150,540),(450,320),(490,310),(900,540)]]))
    h_img=hough_lines(masked_edges,2,np.pi/180,15,40,20)
    final=weighted_img(h_img,image)
    return final   

In [15]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
#%time white_clip.write_videofile(white_output, audio=False)

In [16]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4




  0%|          | 0/682 [00:00<?, ?it/s]

  1%|          | 5/682 [00:00<00:14, 46.28it/s]

  2%|▏         | 11/682 [00:00<00:14, 47.48it/s]

  3%|▎         | 18/682 [00:00<00:13, 50.70it/s]

  3%|▎         | 23/682 [00:00<00:13, 49.28it/s]

  4%|▍         | 28/682 [00:00<00:13, 48.22it/s]

ZeroDivisionError: float division by zero



  4%|▍         | 28/682 [00:10<04:14,  2.57it/s]

In [17]:
yellow_output = 'challenge_test.mp4'
clip2 = VideoFileClip('challenge.mp4')
yellow_clip = clip2.fl_image(process_image)
yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video challenge_test.mp4
[MoviePy] Writing video challenge_test.mp4





  0%|          | 0/251 [00:00<?, ?it/s]


  1%|          | 2/251 [00:00<00:18, 13.72it/s]


  2%|▏         | 5/251 [00:00<00:15, 16.17it/s]


  3%|▎         | 8/251 [00:00<00:14, 17.24it/s]


  4%|▍         | 11/251 [00:00<00:12, 18.58it/s]


  6%|▌         | 14/251 [00:00<00:11, 20.73it/s]


  7%|▋         | 17/251 [00:00<00:10, 22.25it/s]


  8%|▊         | 20/251 [00:00<00:09, 23.75it/s]


  9%|▉         | 23/251 [00:01<00:10, 22.07it/s]


 10%|█         | 26/251 [00:01<00:09, 22.58it/s]


 12%|█▏        | 29/251 [00:01<00:09, 23.69it/s]


 13%|█▎        | 33/251 [00:01<00:08, 25.00it/s]


 14%|█▍        | 36/251 [00:01<00:09, 23.20it/s]


 16%|█▌        | 39/251 [00:01<00:09, 23.35it/s]


 17%|█▋        | 42/251 [00:01<00:08, 24.57it/s]


 18%|█▊        | 45/251 [00:01<00:09, 21.93it/s]


 19%|█▉        | 48/251 [00:02<00:09, 20.62it/s]


 20%|██        | 51/251 [00:02<00:09, 20.77it/s]


 22%|██▏       | 54/251 [00:02<00:10, 17.91it/s]


 22%|██▏       | 56/251 [00:02<00:11, 17

ZeroDivisionError: division by zero




 47%|████▋     | 117/251 [00:23<00:27,  4.88it/s]